In [ ]:
# LOCAL IMPORTS

import importlib
import romanizer
import db

importlib.reload(romanizer)     # Reload romanizer from korean
importlib.reload(db)

In [ ]:
# MODEL DOWNLOAD

import stanza

language = 'ko'
stanza.download(language)       # Download model
nlp = stanza.Pipeline(language) # Initialize neural pipeline

In [ ]:
# DATABASE

import os
from dotenv import load_dotenv
from psycopg2.extensions import connection

load_dotenv(override=True)

conn: connection = db.connect(language)

In [ ]:
# TABLE CREATION

db.create_tables(conn)

In [ ]:
# PHRASE TOKENIZATION
phrase = "2000년 11월 17일이 바로 그런 날이었어요"

doc = nlp(phrase)
dict = doc.to_dict()

print(doc.sentences)

In [ ]:
from color import blue, bold, cyan, green, purple, red


for sentences in dict:
    print("Phrase: ", phrase, '|', blue(romanizer.romanize(phrase)))
    for words in sentences:
        print(bold(cyan("Text: ")), words['text'], '|', blue(romanizer.romanize(words['text'])))

        upos = db.get_upos(conn, words['upos'])
        print(green("Part of Speech: "), upos)

        print(red("Morphemes: "))
        morphemes = words['lemma'].split('+')

        tags = words['xpos'].split('+')
        xpos = db.get_xpos_labels(conn, tags)

        morphemes_info = []
        for morpheme, tag, xpos_label in zip (morphemes, tags, xpos):
            info = db.get_morpheme(conn, morpheme, tag)
            morphemes_info.append(info)
            print(f"    '{morpheme}': '{tag} => {xpos_label.title()}' | {bold(red('Translation'))}: {info['translation']}")



        # for morpheme, tag, xpos_label, info in zip(morphemes, tags, xpos, morphemes_info):
        #     print(f"    '{morpheme}': '{tag} => {xpos_label.title()}' | {bold(red('Translation'))}: {info['translation']}")

        translation = db.get_translation(conn, words['text'], words['upos'], morphemes_info)
        print(purple("Translation: "), translation)
        print("─"*30, '\n')
